<a href="https://colab.research.google.com/github/ddekun/NLP/blob/lesson5/lesson5/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку естественного языка

### Урок 5. Part-of-Speech разметка, NER, извлечение отношений

**Тема «POS-tagger и NER»**

Задание 1. Написать теггер на данных с русским языком

- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить
знание не только букв но и слов

- сравнить все реализованные методы, сделать выводы

In [1]:
!pip install pyconll

In [2]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

import pyconll

In [3]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-09-04 15:24:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M  54.4MB/s    in 0.7s    

2023-09-04 15:27:19 (54.4 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736581/40736581]

--2023-09-04 15:27:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.19

In [4]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [5]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    for word in sent:
        fdata_sent_test.append(word.form)

In [6]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Максимальная длина предложения', MAX_SENT_LEN)
print('Максимальная длина токена', MAX_ORIG_TOKEN_LEN)

Максимальная длина предложения 194
Максимальная длина токена 31


In [7]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

<ipython-input-7-b70f333e48c5>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  unigram_tagger.evaluate(fdata_test)


0.823732013802982

In [8]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

<ipython-input-8-be35a56830c0>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  bigram_tagger.evaluate(fdata_test)


0.8292792499511688

In [9]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
trigram_tagger.evaluate(fdata_test)

<ipython-input-9-1246b3405b88>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  trigram_tagger.evaluate(fdata_test)


0.828550035809623

In [10]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

<ipython-input-10-792eb0e8a39b>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  trigram_tagger.evaluate(fdata_test)


0.8291425222996289

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = backoff_tagger(fdata_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)

tag.evaluate(fdata_test)

<ipython-input-11-4f2b35143c69>:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tag.evaluate(fdata_test)


0.827905462595221

In [12]:
train_token = []
train_label = []
for sent in fdata_train[:]:
    for token in sent:
        train_token.append(token[0])
        train_label.append('NO_TAG' if token[1] is None else token[1])

test_token = []
test_label = []
for sent in fdata_test[:]:
    for token in sent:
        test_token.append(token[0])
        test_label.append('NO_TAG' if token[1] is None else token[1])

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [15]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [16]:
hvectorizer = HashingVectorizer(ngram_range=(1, 4), analyzer='char', n_features=100)

In [17]:
for number, element in enumerate(test_token):
    if element == None:
        test_token[number] = 'Договор'

In [18]:
X_train = hvectorizer.fit_transform(train_token)
X_test = hvectorizer.transform(test_token)

X_train.shape

(426182, 100)

In [19]:
import xgboost
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

In [ ]:
xgb_model.fit(X_train, train_enc_labels)

In [ ]:
pred = xgb_model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
test_enc_labels = le.fit_transform(test_label)

print(classification_report(test_enc_labels, pred))

Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/

- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
   - передаём в сетку токен и его соседей.
   - передаём в сетку только токен.
   - свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/- f1_score.

In [ ]:
!pip install corus
!pip install razdel

In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

In [ ]:
!unzip collection5.zip

In [ ]:
from corus import load_ne5

path_col5 = 'Collection5/'
records = load_ne5(path_col5)

In [ ]:
from razdel import tokenize

words_docs = []
for idx, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [ ]:
import pandas as pd

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

In [ ]:
words_list = []
for word in words_docs:
    words_list.append(word[0])

words_list[:10]

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

nltk_pos_tags = nltk.pos_tag(words_list)

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

chunks = nltk.ne_chunk(nltk_pos_tags, binary=False)

In [ ]:
pos_tag = nltk.pos_tag(words_list)
chunk = nltk.ne_chunk(pos_tag, binary=False)
entities = []
labels = []
for chunk in chunks:
    if hasattr(chunk, 'label'):
        for c in chunk:
            entities.append(c[0])
            labels.append(chunk.label())
    else:
        entities.append(chunk[0])
        labels.append('OUT')

In [ ]:
entities_labels = list(zip(entities, labels))
entities_df = pd.DataFrame(entities_labels, columns = ['Entities', 'Labels'])

entities_df

In [ ]:
df_words['nltk_ner'] = entities_df['Labels']
df_words

In [ ]:
def ner_metrics(df, true_tag, ner_tag, column_name_1, column_name_2):
    right = 0
    fault = 0
    fault_2 = 0

    for index, person in  df.iterrows():
        if (person[column_name_1] == true_tag) & (person[column_name_2] == ner_tag):
            right += 1
        elif (person[column_name_1] == true_tag) & (person[column_name_2] !=  ner_tag):
            fault += 1
        elif (person[column_name_1] != true_tag) & (person[column_name_2] ==  ner_tag):
            fault_2 += 1
    precision = right / (right+fault_2)
    recall = right / (right+fault)
    f1 = (2*precision*recall) / (precision+recall)
    return print(f'precision = {precision}, recall = {recall}, f1_score = {f1}')

In [ ]:
ner_metrics(df_words, 'PER', 'PERSON', 'tag', 'nltk_ner')

In [ ]:
ner_metrics(df_words, 'ORG', 'ORGANIZATION', 'tag', 'nltk_ner')

In [ ]:
!python -m spacy download ru_core_news_md

In [ ]:
import spacy

nlp = spacy.load('ru_core_news_md')

labels=[]
i=0
for word in words_list:
    spacy_ner = nlp(word)
    if spacy_ner.__getitem__(0).ent_type_ == '':
        labels.append('OUT')
    else:
        labels.append(spacy_ner.__getitem__(0).ent_type_)
    i+=1
    if (i % 10000) == 0:
        print(f'обработано {i} слов')

In [ ]:
df_words['spacy_ner'] = labels
df_words

In [ ]:
ner_metrics(df_words, 'PER', 'PER', 'tag', 'spacy_ner')

In [ ]:

ner_metrics(df_words, 'ORG', 'ORG', 'tag', 'spacy_ner')

In [ ]:
ner_metrics(df_words, 'LOC', 'LOC', 'tag', 'spacy_ner')